# handle remote control
In this example we will remotely control a robotic arm using a gamepad controller connected to a web browser machine.


### Create a handle controller
The first thing we need to do is create an instance of the 'Controller' widget, which we will use to drive our Jetbot.
The "Controller" widget accepts an "index" parameter, which specifies the number of controllers. This is useful if you have multiple controllers, or some gamepads come in the form of multiple controllers. Want to use your controller correctly to control
For the robotic arm you need:
1. Open this webpage [http://html5gamepad.com](http://html5gamepad.com).
2. Press the button of the controller you are using
3. Remember the corresponding index number that pops up when you press the button

Next, we will use this index to create and display the controller.

In [ ]:
import ipywidgets.widgets as widgets
controller = widgets.Controller(index=0)  # Replace with the index number of the controller you just tested and are using
display(controller)

Controller()

In [ ]:
# Import library paths
import threading
import time
# Thread operation library
import inspect
import ctypes
# Import robotic arm object
from Arm_Lib import Arm_Device
Arm = Arm_Device()

In [ ]:
# Create a method to actively stop a thread
def _async_raise(tid, exctype):
    """raises the exception, performs cleanup if needed"""
    tid = ctypes.c_long(tid)
    if not inspect.isclass(exctype):
        exctype = type(exctype)
    res = ctypes.pythonapi.PyThreadState_SetAsyncExc(tid, ctypes.py_object(exctype))
    if res == 0:
        raise ValueError("invalid thread id")
    elif res != 1:
        # """if it returns a number greater than one, you're in trouble,
        # and you should call it again with exc=NULL to revert the effect"""
        ctypes.pythonapi.PyThreadState_SetAsyncExc(tid, None)
        
def stop_thread(thread):
    _async_raise(thread.ident, SystemExit)


# Create a method for controlling the movement of the robotic arm using a joystick
## If the simulation mode of the handle is turned on, that is, when the red light is on, the left direction keys cannot be used. Please use the left and right joysticks and the keys on both sides of LR to control the robotic arm.

## Program functions:
1. The left joystick and direction keys control the No. 1 and No. 2 servo. The left and right directions control the left and right movement of the No. 1 servo, and the up and down directions control the forward and backward movement of the No. 2 servo.
2. The right joystick and number keys control the No. 5 and No. 6 servos. The left and right directions control the left and right rotation of the No. 5 servo, and the up and down directions control the clamping and loosening of the No. 6 servo.
3. L1 and L2 control the No. 3 servo to move forward or backward.
4. R1 and R2 control the No. 4 servo to move forward or backward.
5. Press the SELECT button to set all servo angles of the robotic arm to 90 degrees.


In [ ]:
def Arm_Handle():
    s_time = 500
    s_step = 1
    angle_1 = angle_2 = angle_3 = angle_4 =  angle_5 = angle_6 = 90
    while 1:
        # Due to individual differences in joystick controllers, the reset value of the joystick may not necessarily be zero. 
        # Therefore, a threshold of 0.1 is used as a filter to avoid accidental operations.
        # Servo 2, A1 moves up negatively and down positively.
        if controller.axes[1].value <= 0.1 and controller.axes[1].value >= -0.1:
            time.sleep(.000001)
        else:
            if controller.axes[1].value > 0.1:
                angle_2 += s_step
            else:
                angle_2 -= s_step
            if angle_2 > 180:
                angle_2 = 180
            elif angle_2 < 0:
                angle_2 = 0
            Arm.Arm_serial_servo_write(2, angle_2, s_time)
            time.sleep(0.01)

        # Servo 1, A0 moves left negatively and right positively
        if (controller.axes[0].value <= 0.1 and controller.axes[0].value >= -0.1):
                time.sleep(.000001)
        else:
            if controller.axes[0].value > 0.1:
                angle_1 -= s_step
            else:
                angle_1 += s_step
                
            if angle_1 > 180:
                angle_1 = 180
            elif angle_1 < 0:
                angle_1 = 0
            Arm.Arm_serial_servo_write(1, angle_1, s_time)
            time.sleep(0.01)

        # Servo 6, NUM1=B0, NUM3=B2, A2 moves up negatively and down positively
        if controller.buttons[0].value == True:
            angle_6 += s_step
            if angle_6 > 180:
                angle_6 = 180
            elif angle_6 < 0:
                angle_6 = 0
            Arm.Arm_serial_servo_write(6, angle_6, s_time)
            time.sleep(0.01)
        elif controller.buttons[2].value == True:
            angle_6 -= s_step
            if angle_6 > 180:
                angle_6 = 180
            elif angle_6 < 0:
                angle_6 = 0
            Arm.Arm_serial_servo_write(6, angle_6, s_time)
            time.sleep(0.01)
        elif controller.axes[2].value > 0.5:
            angle_6 -= s_step
            if angle_6 > 180:
                angle_6 = 180
            elif angle_6 < 0:
                angle_6 = 0
            Arm.Arm_serial_servo_write(6, angle_6, s_time)
            time.sleep(0.01)
        elif controller.axes[2].value < -0.5:
            angle_6 += s_step
            if angle_6 > 180:
                angle_6 = 180
            elif angle_6 < 0:
                angle_6 = 0
            Arm.Arm_serial_servo_write(6, angle_6, s_time)
            time.sleep(0.01)

        # Servo 5, NUM2=B1, NUM4=B3, A5 moves left negatively and right positively
        if controller.buttons[1].value == True:
            angle_5 += s_step
            if angle_5 > 180:
                angle_5 = 180
            elif angle_5 < 0:
                angle_5 = 0
            Arm.Arm_serial_servo_write(5, angle_5, s_time)
            time.sleep(0.01)
        elif controller.buttons[3].value == True:
            angle_5 -= s_step
            if angle_5 > 180:
                angle_5 = 180
            elif angle_5 < 0:
                angle_5 = 0
            Arm.Arm_serial_servo_write(5, angle_5, s_time)  
            time.sleep(0.01)
        elif controller.axes[5].value > 0.5:
            angle_5 += s_step
            if angle_5 > 180:
                angle_5 = 180
            elif angle_5 < 0:
                angle_5 = 0
            Arm.Arm_serial_servo_write(5, angle_5, s_time)
            time.sleep(0.01)
        elif controller.axes[5].value < -0.5:
            angle_5 -= s_step
            if angle_5 > 180:
                angle_5 = 180
            elif angle_5 < 0:
                angle_5 = 0
            Arm.Arm_serial_servo_write(5, angle_5, s_time)  
            time.sleep(0.01)


        # Servo 4, R1=B5, R2=B7   
        if controller.buttons[5].value == True:
            angle_4 -= s_step
            if angle_4 > 180:
                angle_4 = 180
            elif angle_4 < 0:
                angle_4 = 0
            Arm.Arm_serial_servo_write(4, angle_4, s_time)
            time.sleep(0.01)
        elif controller.buttons[7].value == True:
            angle_4 += s_step
            if angle_4 > 180:
                angle_4 = 180
            elif angle_4 < 0:
                angle_4 = 0
            Arm.Arm_serial_servo_write(4, angle_4, s_time)  
            time.sleep(0.01)

        # Servo 3, L1=B4, L2=B6   
        if controller.buttons[4].value == True:
            angle_3 -= s_step
            if angle_3 > 180:
                angle_3 = 180
            elif angle_3 < 0:
                angle_3 = 0
            Arm.Arm_serial_servo_write(3, angle_3, s_time)
            time.sleep(0.01)
        elif controller.buttons[6].value == True:
            angle_3 += s_step
            if angle_3 > 180:
                angle_3 = 180
            elif angle_3 < 0:
                angle_3 = 0
            Arm.Arm_serial_servo_write(3, angle_3, s_time)  
            time.sleep(0.01)
        
        # Press the SELECT button B8 to set all servos of the robotic arm to 90 degrees
        if controller.buttons[8].value == True:
            angle_1 = angle_2 = angle_3 = angle_4 = angle_5 = angle_6 = 90
            Arm.Arm_serial_servo_write6(90, 90, 90, 90, 90, 90, 1000)
            time.sleep(1)
            

In [ ]:
# Run the code in the cell below to start the thread for real-time control of the robotic arm using the controller.
# Once the controller thread starts, you can control the robotic arm using the controller.
thread2 = threading.Thread(target=Arm_Handle)
thread2.setDaemon(True)
thread2.start()

In [ ]:
# End the controller thread. If this cell is executed, the controller will no longer be able to control the robotic arm.
# If the thread fails to start or stop,
# please restart the kernel and run the steps from the beginning.
stop_thread(thread2)